<a href="https://colab.research.google.com/github/muthu-selvakumar/AIML-course/blob/third_assignment/third_intermediate_assessmeny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

Load data

In [ ]:
train_df = pd.read_csv('/content/train_LZdllcl.csv')
test_df = pd.read_csv('/content/sample_data/test_2umaH9m.csv')
submission_df = pd.read_csv('/content/sample_data/sample_submission_M0L0uXE.csv')

In [ ]:
print(train_df.describe())

In [ ]:
print(train_df.isnull().sum())

In [ ]:
print(train_df['is_promoted'].value_counts(normalize=True))

Pre_processing

In [14]:
imputer_rating = SimpleImputer(strategy='median')
train_df['previous_year_rating'] = imputer_rating.fit_transform(train_df[['previous_year_rating']]).ravel()
test_df['previous_year_rating'] = imputer_rating.transform(test_df[['previous_year_rating']]).ravel()

imputer_edu = SimpleImputer(strategy='most_frequent')
train_df['education'] = imputer_edu.fit_transform(train_df[['education']]).ravel()
test_df['education'] = imputer_edu.transform(test_df[['education']]).ravel()

cat_cols = ['department', 'region', 'education', 'gender', 'recruitment_channel']

for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    # Handle unknown in test
    test_df[col] = test_df[col].apply(lambda x: x if x in le.classes_ else '<Unknown>')
    if '<Unknown>' not in le.classes_:
        le.classes_ = np.append(le.classes_, '<Unknown>')
    test_df[col] = le.transform(test_df[col])

features = ['department', 'region', 'education', 'gender', 'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'KPIs_met >80%', 'awards_won?', 'avg_training_score']

X = train_df[features]
y = train_df['is_promoted']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

num_cols = ['no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'avg_training_score']
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])

# Modeling and fine tuning
scale_pos = (y_train == 0).sum() / (y_train == 1).sum()
xgb = XGBClassifier(random_state=42, scale_pos_weight=scale_pos)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1]
}

grid = GridSearchCV(xgb, param_grid, cv=3, scoring='f1')
grid.fit(X_train, y_train)

best_xgb = grid.best_estimator_
y_pred = best_xgb.predict(X_val)
print("Validation F1 Score:", f1_score(y_val, y_pred))

# Predict on test
X_test = test_df[features]
X_test[num_cols] = scaler.transform(X_test[num_cols])
predictions = best_xgb.predict(X_test)

# Update and save submission
submission_df['is_promoted'] = predictions
submission_df.to_csv('submission.csv', index=False)

Validation F1 Score: 0.392204628501827


/tmp/ipython-input-299013843.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[num_cols] = scaler.transform(X_test[num_cols])


In [15]:
print("Submission saved to submission.csv")

Submission saved to submission.csv
